# Week3_Data Management_2

Today's lab consists of two major parts: 

- (1) Dealing with missing data: Not all datasets are well-cleaned and ready to be used; in fact, this perfect situation is quite rare. When analyzing real-world data, we encounter various issues such as missing data and data type inconsistencies. Today, we will explore several common issues in data management and learn how to address them.
- (2) The integration of multiple datasets, i.e., merging multiple datasets into a single dataset.
    - Data Concatenating multiple Dataframes along the rows
    - Merging dataframes along the columns
  
We will continue to use the Beijing resale housing dataset as an example. 

In [3]:
# We are going to start importing the libraries we need
# In the future, it is a good practice to keep all the imports in one cell so that
# we can easily see what libraries we are using in the notebook.
import pandas as pd

# Part 1. Data cleaning
As you might have already seen, when we work with data, the initial dataset is not always in a shape where we can use it as is. 

Sometimes column names are misspelled or unclear, there may be missing values, or the format of each column is incorrect. Moreoever you may also have noticed that often we can extract information from columns that might make them easier to work with. All these steps can be considered part of a data cleaning process, where we get the dataset ready to be used more effectively for our analysis purposes. 


## 1.1 Getting the data

In [4]:
# load the housing dataset using relative path
df_2012 = pd.read_excel("HouseBeijing2012.xlsx")

In [5]:
# check the column names
df_2012.columns

Index(['HouseID', 'CommunityID', 'TotalPrice', 'TransYear', 'Bedroom',
       'Livingroom', 'Bathroom', 'Size', 'FloorLevel', 'WinSouth',
       'WinSouthNorth', 'Decoration', 'TotalFloor', 'BuiltYear', 'Elevation',
       'Heating', 'TransMonth', 'TransDay', 'District', 'CensusTract',
       'XIAOQUWEB', 'SchQuality', 'NumSubway1km', 'Dist2Subway', 'HospQuality',
       'Dist2Hosp', 'NumHosp1km', 'NumBus200m', 'Dist2CBD', 'Dist2Center'],
      dtype='object')



An interesting question asked by city planners and urban economists is the preference of homebuyers for the relative height of floors. 
   - High-floor dwelling units offer many benefits, such as reduced exposure to traffic pollution and noise, decreased security risks, scenic views, and emotional superiority for being higher up relative to others. [see this paper](https://open.library.ubc.ca/soa/cIRcle/collections/facultyresearchandpublications/52383/items/1.0433565)
   - However, the disadvantages are also apparent, including the vertical commuting costs and the potential hazards of top floors in case of rain leakages, fire, or other emergencies. 
   
So, now we want to explore the relationship between the relative floor level and unit housing price.

In the dataset, we have a column named **FloorLevel**, which should be in 5 levels from 1 to 5. The meaning of these numbers are:
- 1: the ground floor - the very first floor of a building;
- 2: the low floor - the bottom third of a building's total number of floors
- 3: the middle floor - the middle third of a building's total number of floors
- 4: the high floor - the top third of a building's total number of floors
- 5: the top floor - the highest floor

Now let us check this column:

In [7]:
# check the column: FloorLevel
df_2012["FloorLevel"]

0       bottom floor
1                NaN
2                  3
3                  3
4                  4
            ...     
4995    bottom floor
4996               3
4997               3
4998               2
4999       top floor
Name: FloorLevel, Length: 5000, dtype: object

Let us check the column in more details by using: 
   - `Series.unique()`: retrieves all unique values in the column and returns them as a NumPy array.
   - `DataFrame/Series.value_counts()`: Return a Series containing the frequency of each distinct row in the Dataframe.

In [11]:
# check the unique values: 
df_2012["FloorLevel"].unique()

array(['bottom floor', nan, '3', '4', '2', 'top floor'], dtype=object)

The `dtype=object` indicates that the array contains elements of mixed types rather than a single unified type like integers or floats.

In this case, the `FloorLevel` column contains:
- `Strings` (e.g., 'bottom floor', 'top floor').
- `nan` (a floating-point representation of missing data).
- `Strings` representing numbers (e.g., '3', '4').

Next, let us count each unique value in the column uisng `.value_counts()`: 
- we set `dropna=False` ensures that missing values (NaN) are also counted in the result. If set to True (default), NaN values would be excluded from the count.

In [13]:
# count the appearance
df_2012["FloorLevel"].value_counts(dropna=False) 

FloorLevel
3               1839
4               1139
2               1071
top floor        590
bottom floor     358
NaN                3
Name: count, dtype: int64

There are three issues in this column:
   - This column contains `NaN` values, indicating missing data.
   - The column also includes string values, specifically **top floor** and **bottom floor**. We want to assign numeric values: 5 for top floor and 1 for bottom floor.
   - The column includes string values: "2". "3", "4". 

## 1.2 Assessing Data Types
We have no idea about other columns in this dataset. So, one of the next work is to check the data type for each column to make sure that they are in the right format.

In [179]:
# check data type for each column
df_2012.dtypes

HouseID           object
CommunityID        int64
TotalPrice       float64
TransYear          int64
Bedroom            int64
Livingroom         int64
Bathroom           int64
Size             float64
FloorLevel        object
WinSouth           int64
WinSouthNorth      int64
Decoration         int64
TotalFloor         int64
BuiltYear          int64
Elevation          int64
Heating            int64
TransMonth         int64
TransDay           int64
District           int64
CensusTract        int64
XIAOQUWEB         object
SchQuality         int64
NumSubway1km       int64
Dist2Subway      float64
HospQuality        int64
Dist2Hosp        float64
NumHosp1km         int64
NumBus200m         int64
Dist2CBD         float64
Dist2Center      float64
dtype: object

Other columns looks okay. Even if there are columns that goods problematic, I would not necessarily change the data types for all columns (especially when there are a lot), **just the ones that you might potentially need**. 

Now, it seems the only problematic column is `FloorLevel` 

## 1.3 Replacing Data


In this section, we will address the `FloorLevel` column by replacing specific string values and handling missing data.

Summary of Observations:
- 590 observations with the value "top floor" will be replaced with `5`.
- 358 observations with the value "bottom floor" will be replaced with `1`.
- 3 missing (NaN) values will need to be handled by either dropping them or filling them with a specific value.

Let’s start by replacing the string values:

- Replace "top floor" with the value 5.
- Replace "bottom floor" with the value 1.


There are actually a few ways to do this, and let us use a new function `df.replace()`: 
   - `df.replace(to_replace=old_value, value=new_value)`

In [180]:
# replace the string with the corresponding values
## Warning: inplace=True will modify the original column!
df_2012['FloorLevel'].replace('top floor', 5, inplace=True) 
df_2012['FloorLevel'].replace('bottom floor', 1, inplace=True) 

In [181]:
# Let us check the data again: 
df_2012['FloorLevel'].unique()

array([1, nan, '3', '4', '2', 5], dtype=object)

We can also use 
   - `df.loc[df['column_name'] == some_value, 'column_name'] = new_value`

**In-class exercise**

use the `df.loc[]` method to replace "top floor" with value 5; replace "bottom floor" with value 1

In [14]:
df_2012.loc[df_2012['FloorLevel'] == 'top floor','FloorLevel'] = 5
df_2012.loc[df_2012['FloorLevel'] == 'bottom floor','FloorLevel'] = 1

In [15]:
df_2012['FloorLevel'].unique()

array([1, nan, '3', '4', '2', 5], dtype=object)

### 1.4 Null values in pandas. 
In Pandas, you might see three different types of null values appear; `NaN`, `None`, `NA` (only rarely)

- `None`: Represents missing values and is typically used for non-numeric data. In pandas, it is associated with the `object` data type and is often converted to NaN in numeric contexts.
- `NaN` (Not a Number): A special `floating` value in NumPy that represents missing values in numeric columns. It is the standard for missing numeric data in pandas and results in NaN in most mathematical operations.
- `pd.NA`: A pandas-specific null value to handle missing data across all data types. It provides flexibility for numeric, string, or mixed-type columns.

In this dataset, we saw only `NaN` values.
`NaN` (Not a number) is used by Pandas for representing missing data in numeric columns.
- The data type of `NaN` is float
- To detect `NaN`, Pandas provides the `.isna()` and `.notna()` functions.
- Some Pandas operations will generate `NaN`. For example, when we concatenate or merge two DataFrames with different number of columns or keys, the missing columns or rows will be filled with `NaN`.
  

There are a few ways of handling missing data.


### 1.4.1 Removing rows 

We can remove those rows with data missing from a column that we are planning to use in our analysis. 

Here we are going to use the `.isna()` function to check if the `FloorLevel` column has a `NaN`
   - `isna()` returns a boolean (True or False) for each row and we are going to use that boolean to filter the dataframe.

In [18]:
# check the rows with NaN in FloorLevel (.isna())
df_2012[df_2012['FloorLevel'].isna() == True]

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
6,BJCY84112518,2767,2730093.60,2012,3,2,2,132.08,2,1,...,https://bj.lianjia.com/xiaoqu/1111027380518/,0,1,692.53062,9,5122.56097,0,0,11524.68492,16967.87510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,BJTJ84718789,3360,1030038.24,2012,2,1,1,81.84,1,1,...,https://bj.lianjia.com/xiaoqu/1111027382053/,0,0,1669.04965,9,948.21718,1,0,15849.26185,21363.35490
4996,BJFT84287006,736,1300028.16,2012,1,1,1,58.12,3,1,...,https://bj.lianjia.com/xiaoqu/1111027375508/,0,1,770.89444,9,777.47225,1,1,6221.44255,6278.89885
4997,BJDC84781079,200,2190056.96,2012,2,1,1,58.24,3,1,...,https://bj.lianjia.com/xiaoqu/1111027374239/,2,1,715.50723,9,2053.92372,0,0,3779.07141,3505.28158
4998,BJSJ85075781,2133,1500007.54,2012,1,1,1,63.86,2,1,...,https://bj.lianjia.com/xiaoqu/1111027378940/,0,1,863.10517,9,413.21398,2,2,21094.05154,15576.85068


In [17]:
# check the rows without NaN in FloorLevel (.notna())
df_2012[df_2012['FloorLevel'].notna() == True]

# another way: 
df_2012[df_2012['FloorLevel'].isna() == False]

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
6,BJCY84112518,2767,2730093.60,2012,3,2,2,132.08,2,1,...,https://bj.lianjia.com/xiaoqu/1111027380518/,0,1,692.53062,9,5122.56097,0,0,11524.68492,16967.87510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,BJTJ84718789,3360,1030038.24,2012,2,1,1,81.84,1,1,...,https://bj.lianjia.com/xiaoqu/1111027382053/,0,0,1669.04965,9,948.21718,1,0,15849.26185,21363.35490
4996,BJFT84287006,736,1300028.16,2012,1,1,1,58.12,3,1,...,https://bj.lianjia.com/xiaoqu/1111027375508/,0,1,770.89444,9,777.47225,1,1,6221.44255,6278.89885
4997,BJDC84781079,200,2190056.96,2012,2,1,1,58.24,3,1,...,https://bj.lianjia.com/xiaoqu/1111027374239/,2,1,715.50723,9,2053.92372,0,0,3779.07141,3505.28158
4998,BJSJ85075781,2133,1500007.54,2012,1,1,1,63.86,2,1,...,https://bj.lianjia.com/xiaoqu/1111027378940/,0,1,863.10517,9,413.21398,2,2,21094.05154,15576.85068


In [20]:
# We are going to keep only the rows where the data_year column is not a NaN
df_drop = df_2012[df_2012['FloorLevel'].isna()==False]
df_drop

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
6,BJCY84112518,2767,2730093.60,2012,3,2,2,132.08,2,1,...,https://bj.lianjia.com/xiaoqu/1111027380518/,0,1,692.53062,9,5122.56097,0,0,11524.68492,16967.87510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,BJTJ84718789,3360,1030038.24,2012,2,1,1,81.84,1,1,...,https://bj.lianjia.com/xiaoqu/1111027382053/,0,0,1669.04965,9,948.21718,1,0,15849.26185,21363.35490
4996,BJFT84287006,736,1300028.16,2012,1,1,1,58.12,3,1,...,https://bj.lianjia.com/xiaoqu/1111027375508/,0,1,770.89444,9,777.47225,1,1,6221.44255,6278.89885
4997,BJDC84781079,200,2190056.96,2012,2,1,1,58.24,3,1,...,https://bj.lianjia.com/xiaoqu/1111027374239/,2,1,715.50723,9,2053.92372,0,0,3779.07141,3505.28158
4998,BJSJ85075781,2133,1500007.54,2012,1,1,1,63.86,2,1,...,https://bj.lianjia.com/xiaoqu/1111027378940/,0,1,863.10517,9,413.21398,2,2,21094.05154,15576.85068


### 1.4.2 Replacing missing data

We can also replace the missing data with certain values: 
- We can replace the data with the **mean** of the non-NaN column values, for numerical values. (For instance, if our columns were something like "adult heights", then replacing the NaN with the mean values in the columns would allow us to leave the sample mean unchanged, which might be good for regression purposes). 
- We can also replace with the **median** (if you think there are outliers in the sample that might be skewing the mean)
- Replacing with the **mode** (most frequent value) would make more sense if we think that there's some default value 

**What would you do here?**

In [26]:
# get the mode of FloorLevel column
mode_FloorLevel = df_2012['FloorLevel'].mode()[0]
print(mode_FloorLevel)

3


- `.fillna()`: Fill NaN values using the specified method.

In [27]:
# This fills the NaNs with the mode using the .fillna() function
# fillna() is a method that fills in missing values with a value of your choice
df_2012['FloorLevel'].fillna(mode_FloorLevel, inplace = True)

In [28]:
# .unique() and .value_counts 
print(df_2012["FloorLevel"].unique())
print(df_2012["FloorLevel"].value_counts(dropna=False))

[1 '3' '4' '2' 5]
FloorLevel
3    1842
4    1139
2    1071
5     590
1     358
Name: count, dtype: int64


Let us check the data type of the FloorLevel. It is still an Object, because there exist string values like "2", "3", and "4".

In [189]:
# check the data type of the FloorLevel
df_2012["FloorLevel"].dtype

dtype('O')

## 1.5 Changing data types
Notice that we have changed TopFloor and BottomFloor to numeric values, but the column is still showing up as an `object`. Now let's try to convert the string values "2", "3", and "4" to numeric values. 
-  `pd.to_numeric()`: convert all string values to numeric values, see the `errors` parameter to learn how it deal with non-convertible values.
    - `errors` : {'ignore', 'raise', 'coerce'}, default 'raise'
- `.astype()` changes your column types for a particular column.
    - `.astype(int)`: convert a column to an integer type. If the column contains non-numeric values (e.g., strings like "abc") or values that cannot be converted to integers (e.g., "NaN" or None), the operation will raise a ValueError.
    - If the column has missing values (NaN), the conversion will fail because NaN is a floating-point value and cannot be directly converted to an integer.

In [29]:
## .to_numeric()
df_2012['FloorLevel'] = pd.to_numeric(df_2012['FloorLevel'])
print(df_2012['FloorLevel'].dtype)

int64


In [30]:
df_2012.dtypes

HouseID           object
CommunityID        int64
TotalPrice       float64
TransYear          int64
Bedroom            int64
Livingroom         int64
Bathroom           int64
Size             float64
FloorLevel         int64
WinSouth           int64
WinSouthNorth      int64
Decoration         int64
TotalFloor         int64
BuiltYear          int64
Elevation          int64
Heating            int64
TransMonth         int64
TransDay           int64
District           int64
CensusTract        int64
XIAOQUWEB         object
SchQuality         int64
NumSubway1km       int64
Dist2Subway      float64
HospQuality        int64
Dist2Hosp        float64
NumHosp1km         int64
NumBus200m         int64
Dist2CBD         float64
Dist2Center      float64
dtype: object

In [33]:
# .astype(int)
df_2012['FloorLevel'] = df_2012['FloorLevel'].astype(int)

# Part 2. Merging multiple datasets
Specifcially, we will learn: 
- Data Concatenating multiple Dataframes along the rows
- Merging dataframes along the columns

## 2.1 Concatenating multiple Dataframes along the rows

Pandas provides various functions for combining together Series or DataFrame along the rows or columns axis. 
   - `"concatenate along the rows"` means joining DataFrame B to DataFrame A vertically using column names as concatenating/joining identifiers.

Let us try to concatenate the 2012 and 2013 Beijing housing dataset along the rows using `pd.concat()`. 
    To concatenate two DataFrames along the rows, you are requried to check whether the **joining identifiers (column names) are the same in both datasets**. 

Now read the two **cleaned** datasets we want to concatenate and print out their column names:  

### 2.11 Preparing to cancatenate two dataframes

In [2]:
# import pandas as pd if you have not done it yet...
import pandas as pd

In [45]:
# read two cleaned housing datasets
df_2012 = pd.read_excel("HouseBeijing2012_clean.xlsx")
df_2013 = pd.read_excel("HouseBeijing2013_clean.xlsx")

In [46]:
# print out columns names
print(df_2012.columns)
print(df_2013.columns)

Index(['Unnamed: 0', 'HouseID', 'CommunityID', 'TotalPrice', 'TransYear',
       'Bedroom', 'Livingroom', 'Bathroom', 'Size', 'FloorLevel', 'WinSouth',
       'WinSouthNorth', 'Decoration', 'TotalFloor', 'BuiltYear', 'Elevation',
       'Heating', 'TransMonth', 'TransDay', 'District', 'CensusTract',
       'XIAOQUWEB', 'SchQuality', 'NumSubway1km', 'Dist2Subway', 'HospQuality',
       'Dist2Hosp', 'NumHosp1km', 'NumBus200m', 'Dist2CBD', 'Dist2Center',
       'UnitPrice'],
      dtype='object')
Index(['1. HouseID', '2. CommunityID', '3. TotalPrice', '4. TransYear',
       '5. Bedroom', '6. Livingroom', '7. Bathroom', '8. Size',
       '9. FloorLevel', '10. WinSouth', '11. WinSouthNorth', '12. Decoration',
       '13. TotalFloor', '14. BuiltYear', '15. Elevation', '16. Heating',
       '17. TransMonth', '18. TransDay', '20. District', '21. CensusTract',
       '22. XIAOQUWEB', '23. SchQuality', '24. NumSubway1km',
       '25. Dist2Subway', '26. HospQuality', '27. Dist2Hosp', '28. NumHosp

In [47]:
print(len(df_2012.columns),len(df_2013.columns))

32 30


In [48]:
# if it is hard to identify whether the columns are same, we can use: 
list(df_2012.columns) == list(df_2013.columns)

False

In [49]:
# another way: 
a = [col for col in df_2012.columns if col not in df_2013.columns]
a

['Unnamed: 0',
 'HouseID',
 'CommunityID',
 'TotalPrice',
 'TransYear',
 'Bedroom',
 'Livingroom',
 'Bathroom',
 'Size',
 'FloorLevel',
 'WinSouth',
 'WinSouthNorth',
 'Decoration',
 'TotalFloor',
 'BuiltYear',
 'Elevation',
 'Heating',
 'TransMonth',
 'TransDay',
 'District',
 'CensusTract',
 'XIAOQUWEB',
 'SchQuality',
 'NumSubway1km',
 'Dist2Subway',
 'HospQuality',
 'Dist2Hosp',
 'NumHosp1km',
 'NumBus200m',
 'Dist2CBD',
 'Dist2Center',
 'UnitPrice']

Two datasets are different: 
   - two more columns in the df_2012 dataset
   - the existence of ordinal number in the column names of 2013 dataset. 

### 2.12 Concatenate two dataframes

To concatenate two datasets, we need to make sure the column names are the same.
   - Step  1: drop two extra columns - `'Unnamed: 0'` `'UnitPrice'` using [df.drop()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.drop.html)
   - Step 2: assign the column names of the 2012 DataFrame to that of 2013 DataFrame
   - Step 3: apply `.concat()` to combine the two.  

In [50]:
# Step 1: drop two extra columns - 'Unnamed: 0' 'UnitPrice'
df_2012.drop(columns=['Unnamed: 0', 'UnitPrice'], inplace=True)

- again, `inplace = True` modifies the DataFrame in place, meaning it will directly apply the changes to the original DataFrame without needing to assign it to a new variable.

In [51]:
# Step 2: 
# get the column name of the 2012 dataframe
col_2012 = df_2012.columns 

# assign the column names of the 2012 dataset to the column names of 2013 dataset
df_2013.columns = col_2012 

In [57]:
# Step 3: Concate (Join) 2012 and 2013 dataset along the rows using pd.concat() 
df_comb_1213 = pd.concat([df_2012, df_2013], axis = 0)  # axis = 0 is the default
df_comb_1213

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
1,BJCP84958845,2606,1800066.00,2012,3,2,2,129.00,3,0,...,https://bj.lianjia.com/xiaoqu/1111027380050/,0,0,2284.09390,9,9154.80958,0,0,18298.50637,18632.22305
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,BJCY85295028,3506,3400028.52,2013,2,2,2,111.48,2,0,...,https://bj.lianjia.com/xiaoqu/1111027382459/,0,2,718.61206,9,3072.19586,0,0,7698.13101,8724.78408
4996,BJFT86470175,2974,2040030.00,2013,2,1,1,57.00,5,0,...,https://bj.lianjia.com/xiaoqu/1111027381044/,0,0,1096.16421,9,1841.03871,0,0,7833.44901,4972.72498
4997,BJCY85539965,1680,1970005.96,2013,1,1,1,43.94,4,1,...,https://bj.lianjia.com/xiaoqu/1111027377814/,0,1,577.23111,7,292.66932,1,7,5901.86525,6733.63486
4998,BJDX85632173,3293,3150106.60,2013,3,2,2,130.45,1,1,...,https://bj.lianjia.com/xiaoqu/1111027381865/,0,0,1183.40088,8,2576.24539,0,5,11878.33320,12565.01538


Note: The pd.concat() function's argument `axis` controls the concatenation direction:
- `axis=0`: Concatenates along rows (joins two DataFrames vertically). This is the default.
- `axis=1`: Concatenates along columns (joins two DataFrames horizontally).

The combined dataset looks good! It has 10,000 rows by 30 columns. But each row still has the original index that ranges from 0 to 4999. We need to reset the index using the function `df.reset_index(drop, inplace)`. 
   - The arguments `drop = True` means dropping the original index column.
   - The argument `inplace = True` means that the dataframe is modified (reindexed) directly so that we do not need to assign a new variable to store the returned value. 
   - If `inplace = False` or using the default, the dataframe will not change after reindexing, and we have to assign a new variable to save the change

In [58]:
df_comb_1213.reset_index(inplace = True, drop = True)
df_comb_1213

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
1,BJCP84958845,2606,1800066.00,2012,3,2,2,129.00,3,0,...,https://bj.lianjia.com/xiaoqu/1111027380050/,0,0,2284.09390,9,9154.80958,0,0,18298.50637,18632.22305
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,BJCY85295028,3506,3400028.52,2013,2,2,2,111.48,2,0,...,https://bj.lianjia.com/xiaoqu/1111027382459/,0,2,718.61206,9,3072.19586,0,0,7698.13101,8724.78408
9996,BJFT86470175,2974,2040030.00,2013,2,1,1,57.00,5,0,...,https://bj.lianjia.com/xiaoqu/1111027381044/,0,0,1096.16421,9,1841.03871,0,0,7833.44901,4972.72498
9997,BJCY85539965,1680,1970005.96,2013,1,1,1,43.94,4,1,...,https://bj.lianjia.com/xiaoqu/1111027377814/,0,1,577.23111,7,292.66932,1,7,5901.86525,6733.63486
9998,BJDX85632173,3293,3150106.60,2013,3,2,2,130.45,1,1,...,https://bj.lianjia.com/xiaoqu/1111027381865/,0,0,1183.40088,8,2576.24539,0,5,11878.33320,12565.01538


### 2.13 Combining multiple dataframes along rows

Our tasks are: 
   - uses `for loop`, `if statement` and `pd.concat()` to combine the five cleaned housing datasets from 2012 to 2016 to a single DataFrame;
   - export the combined DataFrame to an Excel file.
   
How to design the for loop?
   - We first observe whether the names of these five `Excel files` have any common or different parts. `HouseBeijing2012_clean.xlsx`, `HouseBeijing2013_clean/xlsx`....We see that except for the **year**, all other parts are the same. This is great! We can use a for-loop to iterate over the **year** to read each file into jupyter notebook.
   - We use the `+` operator to concatenate strings and construct the file path for each year:
       - Example: `"HouseBeijing" + str(year) + "_clean.xlsx"`, where `year = "2012",...,"2016"`
       - `str(year)`: Converts the integer year (e.g., 2012) to a string, so it can be concatenated with other string parts of the file path.
       - `+`: Joins the parts of the file path into a single string.
       

In [4]:
# start looping over years
for i in list(range(2012,2017)):
    print(i)                                                         # print year for tracking purpose
    df = pd.read_excel("HouseBeijing" + str(i) + "_clean.xlsx")   # load data for the looping year
    print(df.columns)   

2012
Index(['Unnamed: 0', 'HouseID', 'CommunityID', 'TotalPrice', 'TransYear',
       'Bedroom', 'Livingroom', 'Bathroom', 'Size', 'FloorLevel', 'WinSouth',
       'WinSouthNorth', 'Decoration', 'TotalFloor', 'BuiltYear', 'Elevation',
       'Heating', 'TransMonth', 'TransDay', 'District', 'CensusTract',
       'XIAOQUWEB', 'SchQuality', 'NumSubway1km', 'Dist2Subway', 'HospQuality',
       'Dist2Hosp', 'NumHosp1km', 'NumBus200m', 'Dist2CBD', 'Dist2Center',
       'UnitPrice'],
      dtype='object')
2013
Index(['1. HouseID', '2. CommunityID', '3. TotalPrice', '4. TransYear',
       '5. Bedroom', '6. Livingroom', '7. Bathroom', '8. Size',
       '9. FloorLevel', '10. WinSouth', '11. WinSouthNorth', '12. Decoration',
       '13. TotalFloor', '14. BuiltYear', '15. Elevation', '16. Heating',
       '17. TransMonth', '18. TransDay', '20. District', '21. CensusTract',
       '22. XIAOQUWEB', '23. SchQuality', '24. NumSubway1km',
       '25. Dist2Subway', '26. HospQuality', '27. Dist2Hosp', '2

Okay, now we can read the five files. After checking the column names, we see that the columns for 2013 through 2016 are consistent, while those for 2012 are different. Therefore, we need to make the column names consistent and then using ·pd.concat()·.

In [7]:
import pandas as pd
df_combine = pd.DataFrame()

for i in list(range(2012, 2017)):
    df_temp = pd.read_excel("HouseBeijing" + str(i) + "_clean.xlsx")
    if i == 2012: 
        df_temp.drop(columns = ["Unnamed: 0", "UnitPrice"], inplace = True)
        df_combine = df_temp
        col_2012 = df_temp.columns
    else: 
        df_temp.columns = col_2012
        df_combine = pd.concat([df_combine, df_temp])
df_combine        

,HouseID,CommunityID,TotalPrice,TransYear,Bedroom,Livingroom,Bathroom,Size,FloorLevel,WinSouth,...,XIAOQUWEB,SchQuality,NumSubway1km,Dist2Subway,HospQuality,Dist2Hosp,NumHosp1km,NumBus200m,Dist2CBD,Dist2Center
0,BJFT84326414,1544,1400010.56,2012,2,1,1,69.68,1,0,...,https://bj.lianjia.com/xiaoqu/1111027377493/,0,2,633.24007,9,1803.02071,0,1,9345.20091,7396.31505
1,BJCP84958845,2606,1800066.00,2012,3,2,2,129.00,3,0,...,https://bj.lianjia.com/xiaoqu/1111027380050/,0,0,2284.09390,9,9154.80958,0,0,18298.50637,18632.22305
2,BJDX84905788,2264,1350038.34,2012,2,1,1,88.83,3,1,...,https://bj.lianjia.com/xiaoqu/1111027379274/,0,1,667.21572,8,11158.05983,0,4,22480.82065,20105.06770
3,BJFT00386624,3621,1800006.91,2012,2,1,1,98.69,3,0,...,https://bj.lianjia.com/xiaoqu/1111027382765/,0,1,939.29061,9,1698.79101,0,10,16309.85203,11427.48851
4,BJCY84713854,1127,1970019.58,2012,1,1,1,53.66,4,0,...,https://bj.lianjia.com/xiaoqu/1111027376538/,0,3,476.28267,9,938.35742,2,0,8105.90581,7213.87518
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,101092000000,3341,2840059.18,2016,2,2,1,94.42,5,1,...,https://bj.lianjia.com/xiaoqu/1111027381983/,0,1,868.51668,9,1600.57128,0,2,22187.94046,20070.94098
4996,BJCY91554898,607,2250027.60,2016,1,1,1,48.56,4,0,...,https://bj.lianjia.com/xiaoqu/1111027375189/,0,2,416.47153,9,1395.08991,0,0,7486.44958,6620.63431
4997,101100000000,3619,3930039.20,2016,2,1,1,63.40,5,1,...,https://bj.lianjia.com/xiaoqu/1111027382757/,0,2,218.23739,8,382.18439,1,6,4175.85237,7671.78186
4998,101101000000,1289,2550028.00,2016,3,1,1,89.00,2,1,...,https://bj.lianjia.com/xiaoqu/1111027376948/,0,0,1898.20842,8,11101.69181,0,0,40265.43022,37722.90810


In [9]:
df_combine.reset_index(drop = True, inplace = True)
df_combine.to_excel("Housing_combine.xlsx")

## 2.2 Merging DataFrames along the columns

Now we want to merge DataFrames along the columns. we can use the function `pd.merge()`.
   - `merge along the columns` means merging two dataframes horizontally.
   - Note that the `pd.concat()` can be used to merge along columns by changing the argument `axis = 1`.
   - Also note, the function `pd.merge()` can **ONLY** be used to merge along the columns.
   
In the `df_combine`, you may notice the column (variable) "District" is coded by numbers, 1,2,3... without acknowledging us the specific district names. There is a file called `"DistrictName.xlsx"` in on Canvas storing the district name. We will merge the DataFrame with distrcit name to the current DataFrame. So that we know where a housing record is located.

To merge the two DataFrames, we need to 
   - identify the `merging key (identifier)`, which is the common column existing in both DataFrames.  
   - the merging key in the current DataFrame (df_combine) is the column `"District"` and the merging key in the district DataFrame is the column `"DistrictID"`.  


Check the merging key in the `df_combine`: column "District".

In [10]:
# examine the variable "District"
df_combine["District"]

0        2
1        6
2        4
3        2
4        7
        ..
24995    6
24996    7
24997    7
24998    6
24999    8
Name: District, Length: 25000, dtype: int64

In [11]:
# read the District name file. 
df_district = pd.read_excel("DistrictName.xlsx")
df_district

,DistrictID,DistName_En,DistName_Ch
0,1,Dongcheng,东城
1,2,Fengtai,丰台
2,3,Yizhuang,亦庄
3,4,Daxing,大兴
4,5,Fangshan,房山
5,6,Changping,昌平
6,7,Chaoyang,朝阳
7,8,Haidian,海淀
8,10,Shijingshan,石景山
9,11,Xicheng,西城


In [12]:
# merge the combined data frame with the district name
# left_on: the left merge key, right_on: the right merge key
df_combine = pd.merge(df_combine, df_district, left_on = "District", right_on = "DistrictID")
df_combine.to_excel("Housing_combine.xlsx")

`pd.merge()` is a powerful tool for combining dataframe along the columns, and it has many parameters/arguments.... We are unable to throughly explore all its features and we do not need to. 

   - pd.merge(self, right, how='inner', on=None, left_on=None, right_on=None, left_index=False, right_index=False, sort=False, suffixes=('_x', '_y'), copy=True, indicator=False, validate=None)
 
***`If you have a concatenate/merge task with more requirements (e.g., merging based on multiple keys, or one to many, one to many, many to many, etc.), check tutorial here:` https://pandas.pydata.org/pandas-docs/stable/user_guide/merging.html. `It lists all the possible scenarios of combining DataFrames that can be done using pandas.`***

## 2.3 Slicing string columns

`Key Identifer` is important in merging and concentating multiple dataframes. However, finding the appropriate identifer is not easy. Sometimes, we need to slice the string column, keeping only parts of the string, in order to match with the identifer in another dataframe.

For example, in our dataframe, we have a column `"HouseID"`, storing the unqiue housing transcation number. A typical houseID, such as `BJFT84326414`, consists of two parts: the first part is a four-digit city/district code (e.g., BJFT), and the second part is a unique transaction number for each house. We now only need the transaction number and need to remove the city/district code, or place this part into another column.

To do this, we can: 
- (1) create a new column saving the city/district code
- (2) create another column saving housing transaction number.
- (3) drop the original HouseID

In [36]:
df_2012 = pd.read_excel("HouseBeijing2012_clean.xlsx")

In [38]:
# Extract city and district code (first 4 digits)
df_2012["HouseID"].str[:4]

0       BJFT
1       BJCP
2       BJDX
3       BJFT
4       BJCY
        ... 
4995    BJTJ
4996    BJFT
4997    BJDC
4998    BJSJ
4999    BJCY
Name: HouseID, Length: 5000, dtype: object

In [39]:
# Extract transaction number (remaining part)
df_2012["HouseNo"] = df_2012['HouseID'].str[4:]
df_2012["HouseNo"]

0       84326414 
1       84958845 
2       84905788 
3       00386624 
4       84713854 
          ...    
4995    84718789 
4996    84287006 
4997    84781079 
4998    85075781 
4999    84949599 
Name: HouseNo, Length: 5000, dtype: object

In [40]:
# Drop the original houseID column if needed
df_2012.drop(columns=['HouseID'], inplace=True)

## In-class exercise (Due Feb 6th) 

Download the Manhattan house price data from Canvas. This dataset includes house transaction data from Manhattan from 2011 to 2017. Combine the house price data from each year into a single dataset, and export as an excel/csv file